In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import squeezeformer, transducer
import tensorflow as tf

In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [5]:
config = malaya_speech.config.squeezeformer_s_encoder_config
small_model = squeezeformer.Model(**config)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Subsampling with DS conv
INFO:tensorflow:Initialize block 0
['M', 's', 'C', 's']
INFO:tensorflow:layer type: M
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:fc factor set as 1.0
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:layer type: s
INFO:tensorflow:layer type: C
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:Replace GLU with swish for Conv
INFO:tensorflow:fc factor set as 1.0
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:layer type: s
INFO:tensorflow:Skipping post ln
INFO:tensorflow:Initialize block 1
['M', 's', 'C', 's']
INFO:tensorflow:layer type: M
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:fc factor set as 1.0
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:layer type: s
INFO:tensorflow:layer type: C
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:Replace GLU with swish for Conv
INFO:tensorf

INFO:tensorflow:layer type: s
INFO:tensorflow:layer type: C
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:Replace GLU with swish for Conv
INFO:tensorflow:fc factor set as 1.0
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:layer type: s
INFO:tensorflow:Skipping post ln
INFO:tensorflow:Initialize block 16
['M', 's', 'C', 's']
INFO:tensorflow:layer type: M
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:fc factor set as 1.0
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:layer type: s
INFO:tensorflow:layer type: C
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:Replace GLU with swish for Conv
INFO:tensorflow:fc factor set as 1.0
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:layer type: s
INFO:tensorflow:Skipping post ln
INFO:tensorflow:Initialize block 17
INFO:tensorflow:recovering to dmodel 196, head_size 49
['M', 's', 'C', 's']
INFO:tensorflow:layer type: M
INFO:tensorflow:Use scaling, no preLN
INFO:tensorflow:fc factor set as 1.0
INFO:tensorflow

In [6]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v, 0)
v = tf.expand_dims(v, -1)
v, length_v

(<tf.Tensor 'ExpandDims_3:0' shape=(1, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'ExpandDims_1:0' shape=(1,) dtype=int32>)

In [8]:
config = malaya_speech.config.conformer_small_decoder_config
small_transducer = transducer.rnn.Model(small_model, vocabulary_size = 100, **config)

In [9]:
p = tf.placeholder(tf.int32, [None, None])
p_len = tf.placeholder(tf.int32, [None])

In [10]:
small_logits = small_transducer([v, length_v, p, p_len], training = True)
small_logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



<tf.Tensor 'transducer/transducer_joint/transducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 100) dtype=float32>

In [ ]:
tf.trainable_variables()

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [19]:
%%time

t = [[1,1,1,1,1,1]]
sess.run(small_logits, feed_dict = {i: y, p: t, p_len: [6]}).shape

CPU times: user 2.53 s, sys: 184 ms, total: 2.72 s
Wall time: 2.37 s


(1, 140, 6, 100)